#### Modelado con SVM (Support Vector Machine)

Importacion de librerias y archivo 

In [7]:
import pandas as pd
import numpy as np
from collections import Counter
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import resample

In [8]:
#leer csv
df = pd.read_csv('defunciones_clean.csv')


Quitar columnas no relevantes

In [9]:
# drop columns no relevantes
df.drop('Departamento_Residencia_Madre', axis=1, inplace=True)
df.drop('Año_Registro', axis=1, inplace=True)
df.drop('Mes_Registro', axis=1, inplace=True)
df.drop('Día_Ocurrencia', axis=1, inplace=True)        # No se considera el día de ocurrencia

df.drop('Departamento_Registro', axis=1, inplace=True)

df.drop('Municipio_Registro', axis=1, inplace=True)
df.drop('Municipio_Ocurrencia', axis=1, inplace=True)          
df.drop('Municipio_Residencia_Madre', axis=1, inplace=True)

Separar en entrenaminto y prueba

In [10]:

X = df.drop("Causa_Defunción", axis=1)
Y = df["Causa_Defunción"]

#dividir el dataset en conjunto de entrenamiento y conjunto de testing
X_entreno, X_prueba, y_entreno, y_prueba = train_test_split(X, Y, test_size=0.5, random_state=100)



Se utiiza GridSearchCV para determinar que hiperparámetros son los más óptimos para el respectivo modelo.

In [11]:
from sklearn import svm
from sklearn.metrics import accuracy_score

# Crear el clasificador SVM
clf = svm.SVC(kernel='linear')  # Puedes ajustar el kernel según tus necesidades (lineal, polinómico, RBF, etc.)

# Entrenar el modelo utilizando los datos de entrenamiento
clf.fit(X_entreno, y_entreno)

# Realizar predicciones en el conjunto de prueba
predicciones = clf.predict(X_prueba)

# Calcular el accuracy
accuracy = accuracy_score(y_prueba, predicciones)

# Mostrar el accuracy
print("Accuracy:", accuracy)


Accuracy: 0.8936841569545524
